In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [2]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Optimize Efficacy Tools")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
lm = dspy.LM("gemini/gemini-2.5-pro", temperature=0.9, cache=False)
dspy.settings.configure(lm=lm, track_usage=True)

2025/09/17 16:33:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/17 16:33:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/17 16:33:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [3]:
COMPOUND = "SU11274"

efficacy_agent = CFEfficacyAgent()
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/_internal/_model_construction.py:80: RuntimeWarning: coroutine '_fetch_pubmed_data.<locals>._async_fetch' was never awaited
  def __new__(


Prediction(
    trajectory={'thought_0': 'I need to determine the efficacy of SU11274 in reversing the failing cardiac fibroblast phenotype. My first step is to gather information about the compound. I will start by finding its ChEMBL ID to access its bioactivity and target information in a structured database. This will be the foundation for understanding its mechanism of action.', 'tool_name_0': 'CHEMBL__search_chembl_id', 'tool_args_0': {'goal': 'Find the ChEMBL ID for the compound SU11274 to begin investigating its properties and mechanism of action.', 'query': 'SU11274'}, 'observation_0': "The ChEMBL ID for SU11274 is CHEMBL261641. (Found 1 compound(s) matching 'SU11274': - CHEMBL261641 (None))", 'thought_1': "Now that I have the ChEMBL ID for SU11274 (CHEMBL261641), I can investigate its mechanism of action. Understanding the compound's primary targets is crucial for predicting its effect on cardiac fibroblasts. I will use the `get_mechanisms_of_action` tool to find this informat

In [4]:
steps = len(efficacy_results.trajectory) // 4
cost = sum([x["cost"] for x in lm.history])

print("Steps:", steps)
print("Total Cost (USD):", cost)

Steps: 6
Total Cost (USD): 0.093119375


In [5]:
# tox_agent = ToxicityScreeningAgent(max_iters=12)
# tox_results = tox_agent(compound_name=COMPOUND)
# print(tox_results)

In [6]:
# prioritization_agent = CompoundPrioritizationAgent()
# prioritization_results = prioritization_agent(compound_name=COMPOUND)
# print(prioritization_results)